In [3]:
from SPARQLWrapper import SPARQLWrapper, N3
import rdflib
import requests
import pandas as pd

In [4]:
g_facility = rdflib.Graph()
g_facility.parse("Facility.rdf")

<Graph identifier=N1b645d5ab1084ebda59da22e03f63d47 (<class 'rdflib.graph.Graph'>)>

In [5]:
'''
qres_facility = g_facility.query(
    """
    PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
    PREFIX sites:<http://eunis.eea.europa.eu/rdf/sites-schema.rdf#>
    PREFIX facility:<http://prtr.ec.europa.eu/rdf/schema.rdf#>
    PREFIX geo:<http://www.w3.org/2003/01/geo/wgs84_pos#>
    
    SELECT ?name ?lat ?long
       WHERE {
       ?facility facility:facilityName ?name .
       ?facility facility:inCountry ?country .
       ?facility facility:city "Mannheim" .
       ?facility geo:lat ?lat .
       ?facility geo:long ?long .
       }""")

for row in qres_facility.result:
    print(row[0],row[1],row[2])
    
'''

'\nqres_facility = g_facility.query(\n    """\n    PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>\n    PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>\n    PREFIX sites:<http://eunis.eea.europa.eu/rdf/sites-schema.rdf#>\n    PREFIX facility:<http://prtr.ec.europa.eu/rdf/schema.rdf#>\n    PREFIX geo:<http://www.w3.org/2003/01/geo/wgs84_pos#>\n    \n    SELECT ?name ?lat ?long\n       WHERE {\n       ?facility facility:facilityName ?name .\n       ?facility facility:inCountry ?country .\n       ?facility facility:city "Mannheim" .\n       ?facility geo:lat ?lat .\n       ?facility geo:long ?long .\n       }""")\n\nfor row in qres_facility.result:\n    print(row[0],row[1],row[2])\n    \n'

In [6]:
dbpedia_endpoint = 'http://dbpedia.org/sparql/'


def run_query(dataset, query):
    r = dataset.query(query)
    print('Result of EEA query')
    for row in r.result:
        #print(row)
        yield (row[0], row[1], row[2])
 

def run_query_db_pedia(endpoint, query, vars):
    r = requests.get(endpoint, params={'query': query, 'format': 'application/sparql-results+json', 'encoding':'utf-8'})
    for row in r.json()['results']['bindings']:
       yield [row[v]['value'] for v in vars]
 

def run_all(name):
    queryDbpedia = """
    PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbo:<http://dbpedia.org/ontology/>

    SELECT DISTINCT ?city {
    ?city rdfs:label "%(label)s"@de .
    ?city a dbo:PopulatedPlace .
    }""" % {'label': name}

    query_EEA = """
    PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
    PREFIX sites:<http://eunis.eea.europa.eu/rdf/sites-schema.rdf#>
    PREFIX facility:<http://prtr.ec.europa.eu/rdf/schema.rdf#>

    SELECT ?name ?lat ?long
       WHERE {
       ?facility facility:facilityName ?name .
       ?facility facility:inCountry ?country .
       ?facility facility:city ?place_name .
       ?facility geo:lat ?lat .
       ?facility geo:long ?long .

    FILTER ( regex(?place_name, "%(city1)s" ) )
    }""" % {'city1': name}
    
    try:
        if run_query_db_pedia(dbpedia_endpoint, queryDbpedia, ['city']).__next__():
            print('query result not none')
        
        for city, in run_query_db_pedia(dbpedia_endpoint, queryDbpedia, ['city']):
            print('city is .. {}'.format(city))

            result = pd.DataFrame() 
            for facility, lat, long in run_query(g_facility,  query_EEA):
               #print("\t", facility, lat, long)
                result = result.append({'Facility': facility, 'Longitude': long, 'Latitude': lat}, ignore_index=True)
    except Exception as exp:
        print(exp)
        result = pd.DataFrame() 
        for facility, lat, long in run_query(g_facility,  query_EEA):
       #print("\t", facility, lat, long)
            result = result.append({'Facility': facility, 'Longitude': long, 'Latitude': lat}, ignore_index=True)
        
    
    return result

def main(): 
    #query_city = 'Mannheim'
    #query_city = 'RIVALTA DI TORINO'
    #query_city = 'Rivalta Di Torino'
    query_city = 'Montracol'
    query_result = run_all(query_city)
    print(query_result)
    
    if query_result.empty:
        print('Converting to upper-case...')
        print(query_city.upper())
        query_result = run_all(query_city.upper())
        print(query_result)


if __name__ == "__main__":
    main()

query result not none
city is .. http://dbpedia.org/resource/Montracol
Result of EEA query


/Users/soumya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: accessing the 'result' attribute is deprecated. Iterate over the object instead.
  import sys


Empty DataFrame
Columns: []
Index: []
Converting to upper-case...
MONTRACOL

Result of EEA query
       Facility Latitude Longitude
0  SA Multiporc    46.18      5.11
